In [5]:
import cv2
import os

In [26]:
import cv2
import mediapipe as mp
import numpy as np
mp_holistic = mp.solutions.holistic

def mediapipe_detection(image):
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
        image.flags.writeable = False                  # Image is no longer writeable
        results = holistic.process(image)                 # Make prediction
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
        return image, results

def extract_keypoints(image):
    _, results = mediapipe_detection(image)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [48]:
class_name = 'Delhi'
if not os.path.exists('../data/'+class_name):
    os.mkdir('../data/'+class_name)
vid_path = '../delhi.mp4'
vid = cv2.VideoCapture(vid_path)

frame_count = vid.get(cv2.CAP_PROP_FRAME_COUNT)
i = 0
landmark_list = []
while i < 45:
    ret, frame = vid.read()
    landmarks = extract_keypoints(frame)
    # print(landmarks.shape)
    if not landmarks is None:
        landmark_list.append(landmarks)
        
    # print(frame.shape)
    i += 1

landmark_list = np.array(landmark_list)
print(landmark_list.shape)
np.save('../data/'+class_name+'/1', landmark_list)


(45, 126)


In [49]:
class_labels = {
    'Aeroplane' : 0,
    'Bulb' : 1,
    'Delhi' : 2
}

In [50]:
data_path = '../data/'
train_data = []
labels = []
for i in os.listdir(data_path):
    for j in os.listdir(data_path+i):
        data = np.load(data_path+i+'/'+j)
        labels.append(class_labels[i])
        train_data.append(data)
train_data = np.array(train_data)
labels = np.array(labels)
print(train_data.shape)
print(labels.shape)

(3, 45, 126)
(3,)


In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM

In [52]:
model = Sequential()
model.add(LSTM(128, input_shape=(45, 126), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 45, 128)           130560    
                                                                 
 dropout_1 (Dropout)         (None, 45, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 32)                4128      
                                                                 
 dense_4 (Dense)             (None, 3)                 99        
                                                                 
Total params: 266,371
Trainable params: 266,371
Non-tr

In [53]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [54]:
model.fit(train_data, labels, epochs=100)

Epoch 1/100


UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm/PartitionedCall]] [Op:__inference_train_function_13130]